## Zeitmessungen und Code-Performance 
Wir haben gesehen, dass **viele Problemstellungen auf verschiedene Arten gelöst werden können**, und insbesondere mit verschiedenen Datenstrukturen sowie auf den Datenstrukturen basierenden Methoden. Beispielsweise lassen sich **Mengen auch als Listen** realisieren, wobei wir dabei das einmalige Vorkommen eines Elements "von Hand" sicherstellen müssen. Umgekehrt kann man auch auf die Idee kommen, eine **Liste als Menge von Tupeln** zu realisieren, wobei in jedem Tupel ein Listenelement und seine Position in der Liste gespeichert wird. Man beachte, dass dies trotz des nur einmaligen Vorkommens eines Elements in einer Menge funktioniert.

Die **Kunst der Programmierung**, die im Wesentlichen eine Frage der Erfahrung und Übung darstellt, ist die **Auswahl einer geeigneten Datenstruktur**, und die **Auswahl der "besten" Operationen** auf der Datenstruktur. Beides ist offenbar **stark problemabhängig**.

Das führt auf die generelle Frage, wie wir möglichst **optimale Programme** schreiben können. Gerade zu Beginn der Programmierausbildung fehlt uns aber oft die Erfahrung, eine "gute" Entscheidung a priori zu treffen.

Um unsere Implementierungen nicht nur in diesem Kurs **experimentell zu optimieren**, benötigen wir eine Möglichkeit, die **Laufzeit** unserer Programme oder Teile davon zu messen. Hierbei ist die Laufzeit die Zeit, die der Computer tatsächlich benötigt, um unseren Code auszuführen, und nicht eine irgendwie geratene oder begründete Erwartung.

Um Zeitmessungen durchzuführen, importieren wir das Modul <code>time</code>, welches eine Funktion mit dem Namen <code>perf_counter()</code> bereitstellt. Damit kann die Laufzeit einer Anweisung gemessen werden, analog zu einer Stopuhr. Es empfiehlt sich, den folgenden Code mehrmals auszuführen, um die Schwankungen in der Laufzeit zu bemerken:

In [ ]:
import time

start = time.perf_counter() 
print("Hallo zusammen!")
end = time.perf_counter()

print("Die Anweisung print(\"Hallo zusammen!\") " +
      "benötigt etwa {} Sekunden".format(end - start))


Hallo zusammen!
Die Anweisung print("Hallo zusammen!") benötigt etwa 0.0001933000000917673 Sekunden


Da eine einzige Messung unter Umständen nicht sehr genau ist, bietet es sich an, den Code in einer Schleife mehrfach auszuführen, und den Mittelwert über alle gemessenen Zeiten zu bilden. Dies nennt man **Profiling**. 

In [ ]:
import time
import math

x = 0.5
num_repetitions = 1000000

start = time.perf_counter()
for i in range(num_repetitions):
    sinx = math.sin(x)
end = time.perf_counter()
time = end - start
time_sin = time/num_repetitions

print(("Gesamtzeit: {} Sek.\n" + "Mittelwert für " +
      "einen Schleifendurchlauf: {} Sek.").format(time, time_sin)
) 

Gesamtzeit: 0.1625114000000849 Sek.
Mittelwert für einen Schleifendurchlauf: 1.625114000000849e-07 Sek.


### Beispiel: Zugriffsgeschwindigkeit auf Elemente bei Listen und Mengen
Theoretisch lassen sich Mengenoperationen auch mit Listen darstellen. Das folgende Beispiel zeigt jedoch, dass unter Umständen bestimmte Operationen sehr viel schneller ausgeführt werden können, wenn man mit Mengen arbeitet.

Wir verwenden für aussagekräftigere Ergebnisse den Zufallszahlen-Generator, den wir etwas ad-hoc bereits eingeführt haben.

In [1]:
import time
import random  # Zufallszahlengenerator

# Anlegen einer großen Menge und Liste mit identischen Elementen:
test_list = []
test_set = set()
for k in range(10000):
    number = int(random.random()*1000000)  # ganzzahlige Zufallszahl zwischen 0 und 999999
    # Damit die Mengen auch gleich groß sind, fügen wir
    # number nur dann in test_list ein, wenn die Zahl
    # noch nicht hinzugefügt wurde, also nicht in der Menge ist:
    if number not in test_list:
        test_list.append(number)
    test_set.add(number) # Mengen stellen die Einmaligkeit automatisch sicher
    
print("Anzahl Elemente in test_list =", len(test_list))
print("Anzahl Elemente in test_set  =", len(test_set))

# Messen der Laufzeit:
num_repetitions = 1000

# Menge:
time_set = time.perf_counter()
for i in range(num_repetitions):
    1234 in test_set
time_set = (time.perf_counter() - time_set)/num_repetitions

# Liste:
time_list = time.perf_counter()
for i in range(num_repetitions):
    1234 in test_list
time_list = (time.perf_counter() - time_list)/num_repetitions

# Ausgabe:
print("Laufzeiten für Elementsuche:")
print("Liste: {:10e} Sekunden\nMenge: {:10e} Sekunden\nFaktor Liste/Menge: {:.2f}".format(
    time_list, time_set, time_list/time_set
))

Anzahl Elemente in test_list = 9940
Anzahl Elemente in test_set  = 9940
Laufzeiten für Elementsuche:
Liste: 2.992787e-04 Sekunden
Menge: 3.655390e-07 Sekunden
Faktor Liste/Menge: 818.73


Der durchaus bemerkenswerte große Unterschied kommt zustande, weil Python "unter der Haube" schlauer vorgeht, wenn, wie bei einer Menge, alle Elemente eindeutig sind. Für uns ist an dieser Stelle wichtig, dass wir beim **Entwurf von Programmen** tatsächlich darüber nachgrübeln sollten, welche Datenstruktur "passt", nicht nur im Bezug auf Funktionalität, sondern auch im Bezug auf Laufzeit. Unser Experiment zeigt deutlich, dass eine gute Wahl der Repräsentation (Menge statt Mehrfach-Menge) auch zu einer guten Laufzeit in Python führt. Das beruhigt, aber verschiebt die Verantwortung auf uns als Programmierer\*innen. 


**Fortgeschrittene Erklärung:** Die Elemente einer Menge werden über eine sogenannte [Hash-Tabelle](https://en.wikipedia.org/wiki/Hash_table) gesucht: Zu jedem Wert $x$ in der Menge $A$ wird der Wert $h(x)$ einer möglichst eindeutigen und schnell auswertbaren Funktion $h:A \rightarrow B$ berechnet. Das Element $x$ wird dann an der Position $h(x)$ abgespeichert, das ist sehr ähnlich zu unserem Histogramm-Beispiel weiter oben. Bei der Suche nach einem Element $x$ wird wieder $h(x)$ schnell berechnet, das Ergebnis ist eine "Liste/Menge/sonstwas" der gemäß der Hash-Funktion potentiell infrage kommenden Elemente. Und in dieser kleinen "Liste/Menge/sonstwas" hilft dann noch eine Sortierung.